In [11]:
%load_ext autoreload
%autoreload 2
from motion import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
a = 1
b = 3
c = -2
d = 1
e = -2

def p(t):
  #return np.sin(4 * t)
  return a + b * t + c * t **2 + d * t**3 + e * t**4

In [13]:
num_ts = 10000
t_min = 0
t_max = 10
ts, psns, vels, accs = compute_psn_vel_acc(p, t_min, t_max, num_ts = num_ts)

In [14]:
make_figure(p, t_min, t_max)

In [5]:
fig=go.Figure().set_subplots(4,1, row_heights = [0.1, 0.3, 0.3, 0.3], vertical_spacing = 0.15,
                             specs=[[{"type": "scatter"}], [{"type":"scatter"}], [{"type":"scatter"}], [{"type":"scatter"}]],
                             subplot_titles =  ["foo", "bar", "baz", "fat"])

fig.update_layout(    
    {    
        'xaxis1':{'range': [np.min(psns), np.max(psns)], "title":"position"},
        'yaxis1':{'range': [-1, 1], "visible":False, "zeroline":True},
        'xaxis2':{'range': [np.min(ts) - 1, np.max(ts) + 1]},
        'yaxis2':{'range': [np.min(psns) - 1, np.max(psns) + 1]},
        'xaxis3':{'range': [np.min(ts) - 1, np.max(ts) + 1]},
        'yaxis3':{'range': [np.min(vels) - 1, np.max(vels) + 1]},
        'xaxis4':{'range': [np.min(ts) - 1, np.max(ts) + 1]},
        'yaxis4':{'range': [np.min(accs) - 1, np.max(accs) + 1]},
    }
)

# fig.add_trace(go.Scatter(x= psns, y= np.zeros_like(psns), mode = "markers"), 1, 1)
# fig.add_trace(go.Scatter(x = ts, y = psns, mode = "lines"), 2, 1)
# fig.add_trace(go.Scatter(x = ts, y = vels, mode = "lines"), 3, 1)
# fig.add_trace(go.Scatter(x = ts, y = accs, mode = "lines"), 4, 1)

fig.add_trace(go.Scatter(x = [psns[0]], y= [0], mode = "markers", hoverinfo = "skip"), 1, 1)
fig.add_trace(go.Scatter(x = ts[0:1], y = psns[0:1], mode = "lines", hoverinfo = "skip"), 2, 1)
fig.add_trace(go.Scatter(x = ts[0:1], y = vels[0:1], mode = "lines", hoverinfo = "skip"), 3, 1)
fig.add_trace(go.Scatter(x = ts[0:1], y = accs[0:1], mode = "lines", hoverinfo = "skip"), 4, 1)

frames = [ 
  go.Frame(
    data = [
      go.Scatter(x = [psns[i]], y = [0], mode = "markers", hoverinfo = "skip"),
      go.Scatter(x = ts[0:i+1], y = psns[0:i+1], mode = "lines", hoverinfo = "skip"),
      go.Scatter(x = ts[0:i+1], y = vels[0:i+1], mode = "lines", hoverinfo = "skip"),
      go.Scatter(x = ts[0:i+1], y = accs[0:i+1], mode = "lines", hoverinfo = "skip")
    ],
    name = f"t = {ts[i]}",
    traces = [0, 1, 2, 3]) for i in range(1, num_ts)
]

fig.update(frames = frames)

def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            #"transition": {"duration": duration, "easing": "linear"},
        }

fr_duration=1  # customize this frame duration according to your data!!!!!
sliders = [
            {
                "pad": {"b": 10, "t": 50},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(fr_duration)],
                        "label": f"fr{k+1}",
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]


fig.update_layout(sliders=sliders,
                  updatemenus = [
                        {
                        "buttons": [
                            {
                             "args": [None, frame_args(fr_duration)],
                             "label": "&#9654;", # play symbol
                             "method": "animate",
                            },
                            {
                             "args": [[None], frame_args(fr_duration)],
                             "label": "&#9724;", # pause symbol
                             "method": "animate",
                            }],
                        "direction": "left",
                        "pad": {"r": 10, "t": 70},
                        "type": "buttons",
                        "x": 0.1,
                        "y": 0,
                        }])

fig.write_html("anim.html")